In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm
import re

In [2]:
# 확인할 종목 및 종목 코드
STOCK_DICT = {
    '오리엔트바이오': '002630',
    '윌비스': '008600',
    '태영건설우': '009415',
    '삼보산업': '009620',
    '평화홀딩스': '010770',
    '대영포장': '014160',
    '한솔홈데코': '025750',
    '상지건설': '042940',
    '코나아이': '052400',
    '웹케시': '053580',
    '평화산업': '090080',
    '넥스트아이': '137940',
    '포바이포': '389140'
}

# 수집 시점 및 끝나는 시점
START_DATE = datetime.strptime("2025-05-03", "%Y-%m-%d")
END_DATE = datetime.strptime("2025-06-10", "%Y-%m-%d")

In [3]:
# 3. '브라우저'인 척 하기 위한 헤더 정보
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

# --- 크롤링 실행 부분 ---
total_results = []

# 날짜 형식 변환 함수
def parse_paxnet_date(date_str):
    """팍스넷의 날짜 형식('Thu Jun 26 13:20:02 KST 2025')을 datetime 객체로 변환"""
    try:
        # 'KST' 등 시간대 정보를 제외하고 파싱
        date_parts = date_str.split()
        # 예: ['Thu', 'Jun', '26', '13:20:02', 'KST', '2025']
        # 필요한 부분만 조합: 'Jun 26 2025 13:20:02'
        formatted_str = f"{date_parts[1]} {date_parts[2]} {date_parts[5]} {date_parts[3]}"
        return datetime.strptime(formatted_str, "%b %d %Y %H:%M:%S")
    except Exception:
        return None

In [7]:
for stock_name, stock_code in STOCK_DICT.items():
# for _ in range(1):
#     stock_name, stock_code = list(STOCK_DICT.items())[0]
    print(f"\n====== {stock_name} ({stock_code}) 크롤링 시작 ======")
    
    detail_page_links = []
    
    # 1. 목록 페이지에서 게시물 링크 수집
    for page in range(1, 10): # 최대 10페이지까지 수집
        stop_collecting = False
        try:
            list_url = f"https://www.paxnet.co.kr/tbbs/list?tbbsType=L&id={stock_code}&page={page}"
            res = requests.get(list_url, headers=headers)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, 'html.parser')
            
            # 'comm-list' 내부의 게시물 li만 선택 (광고, 헤더 제외)
            posts = soup.select('ul#comm-list > li:not(.board-col, .board-ad-pc, .board-ad-mobile)')

            if not posts:
                print(f"페이지 {page}에서 더 이상 게시물을 찾을 수 없습니다.")
                break

            for post in posts:
                # 날짜 정보가 있는 span 태그 선택
                date_span = post.select_one('div.date > span.data-date-format')
                if not date_span:
                    continue
                
                # 날짜 추출 및 기간 필터링
                paxnet_date_str = date_span.get('data-date-format')
                post_date = parse_paxnet_date(paxnet_date_str)

                if post_date:
                    if post_date < START_DATE:
                        stop_collecting = True
                        break # 이 페이지의 나머지 게시물은 볼 필요 없음

                    if START_DATE <= post_date <= END_DATE:
                        # 게시물 고유번호(seq) 추출
                        seq_div = post.select_one('div.type')
                        if seq_div and seq_div.has_attr('data-seq'):
                            seq = seq_div['data-seq']
                            # 상세 페이지 URL 조립
                            detail_url = f"https://www.paxnet.co.kr/tbbs/view?id={stock_code}&seq={seq}"
                            detail_page_links.append(detail_url)
            
            print(f"페이지 {page}: 기간 내 게시물 링크 수집 중... (현재 {len(detail_page_links)}개)")
            if stop_collecting:
                print("수집 기간 이전의 게시물에 도달하여 링크 수집을 중단합니다.")
                break
            
            time.sleep(0.5) # 서버 부하 방지

        except Exception as e:
            print(f"목록 페이지 {page} 처리 중 오류: {e}")
            break
            
    # 2. 수집된 링크를 방문하여 상세 정보 수집
    print(f"총 {len(detail_page_links)}개의 게시물 상세 정보를 수집합니다.")
    print(f"수집 사이트 상세 : {detail_page_links}")
    for post_url in tqdm(detail_page_links, desc=f"{stock_name} 상세 정보 수집"):
        try:
            res = requests.get(post_url, headers=headers)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, 'html.parser')

            # --- 상세 정보 추출 (HTML 분석 기반 최종 선택자) ---
            # 제목에서 댓글 수 분리
            title_raw = soup.select_one('div.board-view-tit > h1').get_text(strip=True)
            match = re.match(r'^(.*?)(?:코멘트(\d+))?$', title_raw)
            if match:
                title = match.group(1).strip()
                comment_count = int(match.group(2)) if match.group(2) else 0
            else:
                title = title_raw
                comment_count = 0
            author = soup.select_one('span.nickname').text.strip()
            date_str = soup.select_one('span.data-date-format1')['data-date-format'].split('.')[0] # 'YYYY-MM-DD HH:MM:SS'
            views_text = soup.select_one('span.viewer').text.strip()
            views = views_text.replace('조회', '').strip()
            recommends = soup.select_one('span#recommendCount').text.strip()
            
            # 내용에서 불필요한 부분(공유/추천 버튼 등) 제거 후 텍스트 추출
            content_div = soup.select_one('div#bbsWrtCntn')
            if content_div:
                # div 내의 'board-view-func' 클래스를 가진 div 제거
                func_div = content_div.find('div', class_='board-view-func')
                if func_div:
                    func_div.decompose()
                content = content_div.get_text('\n', strip=True) # 줄바꿈 유지하며 텍스트 추출
            else:
                content = ""

            total_results.append({
                '종목명': stock_name,
                '날짜': date_str.split(' ')[0], # YYYY-MM-DD
                '시간': date_str.split(' ')[1], # HH:MM:SS
                '제목': title,
                '댓글수' : comment_count,
                '작성자': author,
                '조회수': int(views.replace(',', '')),
                '추천수': int(recommends.replace(',', '')),
                '내용': content,
                'URL': post_url
            })
            
            time.sleep(0.3)
        
        except Exception as e:
            # print(f"상세 페이지 처리 중 오류: {e} (URL: {post_url})")
            continue



====== 오리엔트바이오 (002630) 크롤링 시작 ======
페이지 1: 기간 내 게시물 링크 수집 중... (현재 1개)
수집 기간 이전의 게시물에 도달하여 링크 수집을 중단합니다.
총 1개의 게시물 상세 정보를 수집합니다.
수집 사이트 상세 : ['https://www.paxnet.co.kr/tbbs/view?id=002630&seq=150357589326523']


오리엔트바이오 상세 정보 수집: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]



====== 윌비스 (008600) 크롤링 시작 ======
페이지 1: 기간 내 게시물 링크 수집 중... (현재 6개)
수집 기간 이전의 게시물에 도달하여 링크 수집을 중단합니다.
총 6개의 게시물 상세 정보를 수집합니다.
수집 사이트 상세 : ['https://www.paxnet.co.kr/tbbs/view?id=008600&seq=150357589332165', 'https://www.paxnet.co.kr/tbbs/view?id=008600&seq=150357589285678', 'https://www.paxnet.co.kr/tbbs/view?id=008600&seq=150357589280676', 'https://www.paxnet.co.kr/tbbs/view?id=008600&seq=150357589279937', 'https://www.paxnet.co.kr/tbbs/view?id=008600&seq=150357589269036', 'https://www.paxnet.co.kr/tbbs/view?id=008600&seq=150357589261292']


윌비스 상세 정보 수집: 100%|██████████| 6/6 [00:04<00:00,  1.29it/s]



====== 태영건설우 (009415) 크롤링 시작 ======
페이지 1: 기간 내 게시물 링크 수집 중... (현재 0개)
수집 기간 이전의 게시물에 도달하여 링크 수집을 중단합니다.
총 0개의 게시물 상세 정보를 수집합니다.
수집 사이트 상세 : []


...

평화산업 상세 정보 수집: 100%|██████████| 6/6 [00:04<00:00,  1.24it/s]



====== 넥스트아이 (137940) 크롤링 시작 ======
페이지 1: 기간 내 게시물 링크 수집 중... (현재 12개)
수집 기간 이전의 게시물에 도달하여 링크 수집을 중단합니다.
총 12개의 게시물 상세 정보를 수집합니다.
수집 사이트 상세 : ['https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589284252', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589284238', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589284064', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589284062', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589283947', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589283945', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589283914', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589283911', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589283890', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589283888', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589267667', 'https://www.paxnet.co.kr/tbbs/view?id=137940&seq=150357589265541']


넥스트아이 상세 정보 수집: 100%|██████████| 12/12 [00:07<00:00,  1.66it/s]



====== 포바이포 (389140) 크롤링 시작 ======
페이지 1: 기간 내 게시물 링크 수집 중... (현재 28개)
페이지 2: 기간 내 게시물 링크 수집 중... (현재 33개)
수집 기간 이전의 게시물에 도달하여 링크 수집을 중단합니다.
총 33개의 게시물 상세 정보를 수집합니다.
수집 사이트 상세 : ['https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589337524', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589328035', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589327966', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589327517', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589327424', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589327160', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589326679', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589326586', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589326477', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589326457', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589326393', 'https://www.paxnet.co.kr/tbbs/view?id=389140&seq=150357589326130', 'htt

포바이포 상세 정보 수집: 100%|██████████| 33/33 [00:24<00:00,  1.37it/s]


In [8]:
# 3. 최종 결과를 DataFrame으로 변환 후 CSV 저장
if total_results:
    df = pd.DataFrame(total_results)
    df.sort_values(by='날짜', inplace=True) 
    file_name = f"팍스넷_21대_대선_테마주_크롤링_{datetime.now().strftime('%Y%m%d')}.csv"
    df.to_csv(file_name, index=False, encoding='utf-8-sig')
    print(f"\n크롤링 완료! 총 {len(total_results)}개의 데이터를 '{file_name}' 파일로 저장했습니다.")
else:
    print("\n수집된 데이터가 없습니다.")


크롤링 완료! 총 177개의 데이터를 '팍스넷_21대_대선_테마주_크롤링_20250627.csv' 파일로 저장했습니다.
